In [8]:
#library

import torch
import numpy as np
import torch.optim as optim

In [2]:
torch.manual_seed(0)

In [3]:
input_size = 4
hidden_size = 2

In [4]:
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

input_data_np = np.array([[h,e,l,l,o],[e,o,l,l,l],[l,l,e,e,l]],dtype = np.float32)

In [5]:
input_data = torch.Tensor(input_data_np)

# torch RNN

In [6]:
rnn = torch.nn.RNN(input_size, hidden_size)

outputs, _status = rnn(input_data)

print(outputs)
print(outputs.size())

tensor([[[-0.7497, -0.6135],
         [-0.5282, -0.2473],
         [-0.9136, -0.4269],
         [-0.9136, -0.4269],
         [-0.9028,  0.1180]],

        [[-0.5753, -0.0070],
         [-0.9052,  0.2597],
         [-0.9173, -0.1989],
         [-0.9173, -0.1989],
         [-0.8996, -0.2725]],

        [[-0.9077, -0.3205],
         [-0.8944, -0.2902],
         [-0.5134, -0.0288],
         [-0.5134, -0.0288],
         [-0.9127, -0.2222]]], grad_fn=<StackBackward0>)
torch.Size([3, 5, 2])


In [7]:
print(_status) #hidden vector

tensor([[[-0.9077, -0.3205],
         [-0.8944, -0.2902],
         [-0.5134, -0.0288],
         [-0.5134, -0.0288],
         [-0.9127, -0.2222]]], grad_fn=<StackBackward0>)


# torch RNN training

In [9]:
char_set = ['h','i','e','l','o']

input_size = len(char_set)

hidden_size = len(char_set)

learning_rate = 0.1

In [10]:
x_data = [[0,1,0,2,3,3]]

x_one_hot = [[[1,0,0,0,0],
              [0,1,0,0,0],
              [1,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,1,0],
              [0,0,0,1,0]]]

y_data = [[1,0,2,3,3,4]]

In [11]:
x = torch.FloatTensor(x_one_hot)
y = torch.LongTensor(y_data)

In [12]:
rnn = torch.nn.RNN(input_size, hidden_size, batch_first = True)

In [13]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

In [14]:
#training

for i in range(100):

    optimizer.zero_grad()

    outputs, _status = rnn(x)

    loss = criterion(outputs.view(-1,input_size), y.view(-1))

    loss.backward()

    optimizer.step()

    result = outputs.data.numpy().argmax(axis = 2)

    result_str = ''.join([char_set[c] for c in np.squeeze(result)])

    print(i,'loss:', loss.item(), 'prediction:', result, 'true y:', y_data, 'prediction str:', result_str)

0 loss: 1.6659356355667114 prediction: [[3 2 3 2 2 2]] true y: [[1, 0, 2, 3, 3, 4]] prediction str: leleee
1 loss: 1.3435620069503784 prediction: [[3 3 3 3 3 3]] true y: [[1, 0, 2, 3, 3, 4]] prediction str: llllll
2 loss: 1.1607609987258911 prediction: [[3 3 3 3 3 3]] true y: [[1, 0, 2, 3, 3, 4]] prediction str: llllll
3 loss: 1.0613101720809937 prediction: [[3 3 2 3 3 3]] true y: [[1, 0, 2, 3, 3, 4]] prediction str: llelll
4 loss: 1.0018833875656128 prediction: [[3 3 2 3 3 3]] true y: [[1, 0, 2, 3, 3, 4]] prediction str: llelll
5 loss: 0.9612741470336914 prediction: [[3 3 2 3 3 3]] true y: [[1, 0, 2, 3, 3, 4]] prediction str: llelll
6 loss: 0.9296206831932068 prediction: [[3 3 2 3 3 3]] true y: [[1, 0, 2, 3, 3, 4]] prediction str: llelll
7 loss: 0.900706946849823 prediction: [[3 3 2 3 3 4]] true y: [[1, 0, 2, 3, 3, 4]] prediction str: llello
8 loss: 0.8716586232185364 prediction: [[3 3 2 3 3 4]] true y: [[1, 0, 2, 3, 3, 4]] prediction str: llello
9 loss: 0.840173065662384 prediction: 

# long sentence

In [15]:
sample = ' if you want you'

In [16]:
char_set = list(set(sample))

char_dic = {c: i for i,c in enumerate(char_set)}

print(char_dic)

{'t': 0, 'o': 1, 'u': 2, 'a': 3, 'y': 4, 'i': 5, 'n': 6, 'f': 7, 'w': 8, ' ': 9}


In [18]:
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

In [20]:
sample_idx = [char_dic[c] for c in sample]

x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

In [21]:
x = torch.FloatTensor(x_one_hot)
y = torch.LongTensor(y_data)

<ipython-input-21-0520fb99e116>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  x = torch.FloatTensor(x_one_hot)


In [22]:
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first = True)

In [23]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

In [24]:
#training

for i in range(50):

    optimizer.zero_grad()

    outputs, _status = rnn(x)

    loss = criterion(outputs.view(-1, dic_size), y.view(-1))

    loss.backward()

    optimizer.step()

    result = outputs.data.numpy().argmax(axis = 2)

    result_str = ''.join([char_set[c] for c in np.squeeze(result)])

    print(i, 'loss:', loss.item(), 'prediction:', result, 'true y:', y_data, 'prediction str:', result_str)

0 loss: 2.2741920948028564 prediction: [[9 0 7 9 9 7 0 9 7 6 0 7 9 2 7]] true y: [[5, 7, 9, 4, 1, 2, 9, 8, 3, 6, 0, 9, 4, 1, 2]] prediction str:  tf  ft fntf uf
1 loss: 2.0447263717651367 prediction: [[9 2 9 9 9 2 9 9 3 9 2 9 9 9 2]] true y: [[5, 7, 9, 4, 1, 2, 9, 8, 3, 6, 0, 9, 4, 1, 2]] prediction str:  u   u  a u   u
2 loss: 1.7993568181991577 prediction: [[9 2 9 4 1 2 9 2 3 9 2 9 4 9 2]] true y: [[5, 7, 9, 4, 1, 2, 9, 8, 3, 6, 0, 9, 4, 1, 2]] prediction str:  u you ua u y u
3 loss: 1.6393059492111206 prediction: [[4 2 9 4 1 2 9 4 3 9 0 9 4 1 2]] true y: [[5, 7, 9, 4, 1, 2, 9, 8, 3, 6, 0, 9, 4, 1, 2]] prediction str: yu you ya t you
4 loss: 1.4978060722351074 prediction: [[4 1 9 4 1 2 9 4 3 9 0 9 4 1 2]] true y: [[5, 7, 9, 4, 1, 2, 9, 8, 3, 6, 0, 9, 4, 1, 2]] prediction str: yo you ya t you
5 loss: 1.3884620666503906 prediction: [[4 1 9 4 1 2 9 8 3 9 0 9 4 1 2]] true y: [[5, 7, 9, 4, 1, 2, 9, 8, 3, 6, 0, 9, 4, 1, 2]] prediction str: yo you wa t you
6 loss: 1.2922332286834717 predict

# very long sentence

In [25]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [26]:
sentence

"if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."

In [27]:
char_set = list(set(sentence))

char_dic = {c: i for i,c in enumerate(char_set)}

In [28]:
dic_size = len(char_dic)

hidden_size = len(char_dic)

sequence_length = 10

learning_rate = 0.1

In [34]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):

    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i+1:i + sequence_length+1]

    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [35]:
x = torch.FloatTensor(x_one_hot)

y = torch.LongTensor(y_data)

In [36]:
class Net(torch.nn.Module):

    def __init__(self, input_dim, hidden_dim, layers):

        super(Net,self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers,
                                batch_first = True)

        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True)


    def forward(self,x):

        x,_status = self.rnn(x)

        x = self.fc(x)

        return x

net = Net(dic_size, hidden_size, 2)

In [37]:
criterion = torch.nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), learning_rate)

In [38]:
#training

for i in range(100):

    optimizer.zero_grad()

    outputs = net(x)

    loss = criterion(outputs.view(-1,dic_size), y.view(-1))

    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim = 2)

    predict_str = ''

    for j, result in enumerate(results):

        if j == 0:

            predict_str += ''.join([char_set[t] for t in result])

        else:

            predict_str += char_set[result[-1]]

    print(predict_str)


,nn,nnnnnn,n,nnnn, nnnnnn,nnnnnnnnnnnnnnnnnn,nnntn,nnnnn,nn,nnnnnntnnnnnnnnnnnnnnnnnnnnnnnnn,n,nn,nnnnnnnnnnnnnnnnn,nnn,n,nn,,nnnnnn,nn,nnnnnnnnnnn,n,ntnnnnnnn,nnnnnnnnnnnnnn,nnnn
                                                                                                                                                                                   
mm.oem.otmmmmemmmemmmmmmmmemmmmmmmmmmmmmemmmmmmmmmppmmmyemyyemmyemmmmmmmmmmmemmmemmytemmmmemmymmmmemm.tmmmmmemm.emmmmmmmmmmmyemmytmmmmmyemmyemmmmmmmeemypmmymmmmmmmmemm.mmmmmmmmmmm
thherhdhorrrerrreeeeereueeeeeerreeerreeeereerrreearererereerrreeeeeereereeeerrreeredeerreeererreeerrureerureareueereeereeureeerreerereeerrreeerereeereeerreureeeeeeerreearreerreerr
t oaret oaaaaaaaaaaaaaaraaooaaaraaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaraaaaaaaaaaaaaaaaaaaaraaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
  o cle                        d                  o        o  o      o     o                 d      